<a href="https://colab.research.google.com/github/Sai23d/CS345_FinalProject/blob/main/DSCI521_Final_Project_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### College of Computing and Informatics, Drexel University
### DSCI 521: Data Analysis and Interpretation
---

## Final Project Report

## Project Title:

## Student(s):

## Date:
---

## Your project should include the following components:
- Problem Definition: Define a clear problem or task to solve using data analysis techniques.

- Dataset Selection: You must choose a dataset relevant to your interests or a specific domain. The dataset should be of sufficient size and complexity to demonstrate various data analysis techniques.

- Exploratory Data Analysis (EDA): You need to perform thorough EDA on the dataset to understand its characteristics, identify patterns, missing values, outliers, and potential features for modeling.

- Feature Engineering: Implement feature engineering for creating new features, transforming existing ones, or selecting relevant features.

- Model Evaluation and Selection: Experiment with different data analysis algorithms and techniques. Evaluate the models using appropriate evaluation metrics

- Conclusion: Discuss your findings and future work.

## You should write the report with following characristics to ensure effective communication:
- Visualization and Interpretation: You should use visulization throughout EDA, modeling, and evaluation to illustrate any insights.

- Code and Implementation: Throughout the notebook, you must write well-structured and commented code.

- Documentation and Presentation: Throughout the notebook, you must add comprehensive commentary text to explain plots and code snippets. Always provide interpretations and explanations to document your analyses and results.

### 1. Problem Definition
---
*(Define the problem that will be solved in this data analytics project.)*

This project focuses on the key question:

How do socioeconomic factors affect access to the internet and digital devices across U.S. communities? While we know that digital access is often worse in disadvantaged areas, this project uses recent data from the American Community Survey (ACS) 5-Year Estimates to measure these patterns more precisely.(2018-2022) We will: Identify areas (census tracts) with low digital access, such as households without internet or computers. Explore how factors like income, education, and housing costs relate to digital access. Build models to predict which communities are most likely to face digital exclusion and which factors matter most. Our goal is to better understand how inequality impacts digital access and provide useful insights for decision-makers working to close the digital gap.

### 2. Data Sets
---
*(Describe the origin of the data sources. What is the format of the original data? How to access the data?)*

The primary dataset used in this project comes from the American Community Survey (ACS) 5-Year Estimates, conducted by the U.S. Census Bureau. The ACS is an ongoing nationwide survey that collects detailed population and housing information from approximately 3.5 million addresses each year. The 5-year estimates aggregate data over five years to provide reliable statistics at small geographic levels such as census tracts and other geographic levels.

In this project, we focus on variables related to:

Digital access (e.g., internet subscriptions, device availability) Socioeconomic status (e.g., household income, education level, rent burden) Demographic context (e.g., total population, age, race) These data are widely used in policy planning, academic research, and social service allocation.

Format of the Data: The ACS data are provided in multiple formats:

CSV (Comma-Separated Values) files — for individual tables (e.g., B28002, B19013) API Access — via the U.S. Census Bureau API for automated querying

Each dataset includes:

A GEOID column (unique geographic identifier) Estimate and Margin of Error (MOE) columns for each variable Metadata, including variable labels and geographic definitions

There are several ways to access ACS 5-Year Estimates:

Census Bureau Website (https://www2.census.gov/programs-surveys/acs/summary_file/) Use the “Advanced Search” to filter by year, geographic level, and table ID (e.g., B28002 for internet access) Download selected tables as CSV files

U.S. Census Bureau API Register for a free API key at https://api.census.gov/data/key_signup.html

In [4]:
# ======================================
# American Community Survey (ACS) 5-Year Estimates
# Project: Digital Divide in U.S. Communities
# Year: 2018-2022
# ======================================

# -----------------------------
# 0. Install required packages
# -----------------------------
!pip install census us pandas matplotlib seaborn scikit-learn

# -----------------------------
# 1. Import libraries
# -----------------------------
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from census import Census
from us import states
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import numpy as np

# -----------------------------
# 2. Connect to the Census API
# -----------------------------
API_KEY = "0b77d873815d425f3098f0c0aa6be4a000493aae"
c = Census(API_KEY)

year = 2022  # ACS 5-Year Estimates (2018-2022)

# -----------------------------
# 3. Define variables to pull
# -----------------------------
variables = {
    "B28002_013E": "no_internet",        # Households without internet
    "B28001_002E": "with_computer",      # Households with computers
    "B19013_001E": "median_income",      # Median household income
    "B15003_017E": "high_school",        # High school grads
    "B15003_022E": "bachelors",          # Bachelor's degree
    "B25070_001E": "gross_rent",         # Rent burden (total households paying rent)
    "B01003_001E": "total_population",   # Total population
}

# -----------------------------
# 4. Pull data for all states and tracts
# -----------------------------
all_data = []

for state in states.STATES:
    print(f"Fetching data for {state.name}...")
    try:
        data = c.acs5.state_county_tract(
            list(variables.keys()),
            state.fips,
            "*",  # all counties
            "*",  # all tracts
            year=year
        )
        all_data.extend(data)
    except Exception as e:
        print(f"Error fetching {state.name}: {e}")

# Convert to DataFrame
df = pd.DataFrame(all_data)

# -----------------------------
# 5. Create GEOID20 for unique tract ID
# -----------------------------
df['GEOID20'] = df['state'] + df['county'] + df['tract'].str.zfill(6)

Fetching data for Alabama...
Fetching data for Alaska...
Fetching data for Arizona...
Fetching data for Arkansas...
Fetching data for California...
Fetching data for Colorado...
Fetching data for Connecticut...
Fetching data for Delaware...
Fetching data for Florida...
Fetching data for Georgia...
Fetching data for Hawaii...
Fetching data for Idaho...
Fetching data for Illinois...
Fetching data for Indiana...
Fetching data for Iowa...
Fetching data for Kansas...
Fetching data for Kentucky...
Fetching data for Louisiana...
Fetching data for Maine...
Fetching data for Maryland...
Fetching data for Massachusetts...
Fetching data for Michigan...
Fetching data for Minnesota...
Fetching data for Mississippi...
Fetching data for Missouri...
Fetching data for Montana...
Fetching data for Nebraska...
Fetching data for Nevada...
Fetching data for New Hampshire...
Fetching data for New Jersey...
Fetching data for New Mexico...
Fetching data for New York...
Fetching data for North Carolina...
Fetc

### 3. Exploration and Feature Engineering
---
*(Describe and present any code and methods used for exploring and visualizing the data, including statistical analysis and examination of correlations between features. Perform feature engineering to support model development.)*

In [ ]:
# ---- 6) Rename columns and clean ----
# Make sure your `variables` dict includes these denominators:
# "B28002_001E": "hh_total_inet",
# "B28001_001E": "hh_total_comp",
# "B15003_001E": "edu_total_25p",
df = df.rename(columns={k:v for k,v in variables.items() if k in df.columns}).copy()

# numeric casting
num_cols = [v for v in variables.values() if v in df.columns]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# GEOID hygiene (if not already done)
df["tract"] = df["tract"].astype(str).str.zfill(6)
df["GEOID"] = df["state"] + df["county"] + df["tract"]

# Handle sentinel codes (rare, but safe)
df.replace({-666666666: np.nan, -6666666666: np.nan, np.inf: np.nan, -np.inf: np.nan}, inplace=True)

# ---- 7) Derived variables (correct denominators) ----
# % households with no internet
if {"no_internet","hh_total_inet"}.issubset(df.columns):
    df["pct_no_internet"] = (df["no_internet"] / df["hh_total_inet"]).clip(0, 1)

# % households with a computer (optional but useful)
if {"with_computer","hh_total_comp"}.issubset(df.columns):
    df["pct_with_computer"] = (df["with_computer"] / df["hh_total_comp"]).clip(0, 1)

# Education shares for 25+ population
if {"edu_total_25p","high_school"}.issubset(df.columns):
    df["hs_grad_share"] = (df["high_school"] / df["edu_total_25p"]).clip(0, 1)
if {"edu_total_25p","bachelors"}.issubset(df.columns):
    df["bachelor_share"] = (df["bachelors"] / df["edu_total_25p"]).clip(0, 1)

# ---- 8) Enhanced summary statistics ----
def enhanced_summary(df, cols):
    out = {}
    for c in cols:
        if c in df.columns:
            s = df[c]
            out[c] = {
                "count": s.count(),
                "missing": s.isna().sum(),
                "mean": s.mean(),
                "std": s.std(),
                "min": s.min(),
                "25%": s.quantile(0.25),
                "50%": s.median(),
                "75%": s.quantile(0.75),
                "max": s.max(),
                "zeros": (s == 0).sum() if s.notna().any() else np.nan,
            }
    return pd.DataFrame(out).T

summary_cols = list({
    "median_income","total_population","hh_total_inet","hh_total_comp",
    "no_internet","with_computer","pct_no_internet","pct_with_computer",
    "hs_grad_share","bachelor_share"
} & set(df.columns))  # keep only those that exist

summary_df = enhanced_summary(df, summary_cols)
display(summary_df.round(3))


In [6]:
import numpy as np, pandas as pd
from us import states

# 0) If your df still has raw ACS codes, rename using your 'variables' dict
# (Make sure your variables dict includes the denominators below)
variables = {
    "B28002_001E": "hh_total_inet",      # <-- add this
    "B28002_013E": "no_internet",
    "B28001_001E": "hh_total_comp",      # <-- and this
    "B28001_002E": "with_computer",
    "B01003_001E": "total_population",
    # ... any others you pulled
}
# Only rename columns that are present
df = df.rename(columns={k:v for k,v in variables.items() if k in df.columns}).copy()

# 1) Make sure types are numeric
for c in ["no_internet","with_computer","hh_total_inet","hh_total_comp","total_population"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# 2) Quick sanity check: what’s still missing?
needed_for_pct = {"no_internet","hh_total_inet"}
missing = needed_for_pct - set(df.columns)
print("Missing needed columns:", missing)

if missing:
    raise KeyError(
        f"You're missing {missing}. Go back to the data-pull cell and include "
        f"ACS codes B28002_013E (no_internet) AND B28002_001E (hh_total_inet). "
        f"Re-run the pull, then re-run this cell."
    )

# 3) Aggregate by state (weighted % = sum(numerators)/sum(denominators))
fips2name = {s.fips: s.name for s in states.STATES}
state_summary = (
    df.groupby("state", dropna=False)
      .agg(no_internet   = ("no_internet",   "sum"),
           hh_total_inet = ("hh_total_inet", "sum"),
           with_computer = ("with_computer","sum") if "with_computer" in df.columns else ("no_internet","sum"),
           hh_total_comp = ("hh_total_comp","sum") if "hh_total_comp" in df.columns else ("no_internet","sum"),
           total_population=("total_population","sum") if "total_population" in df.columns else ("no_internet","sum"))
      .reset_index()
)

state_summary["state_name"] = state_summary["state"].map(fips2name)
state_summary["Pct_No_Internet"] = 100 * state_summary["no_internet"] / state_summary["hh_total_inet"]
state_summary = state_summary.sort_values("Pct_No_Internet", ascending=False)

cols_show = ["state_name","Pct_No_Internet","no_internet","hh_total_inet"]
print("Top 5 States with Highest % of Households Without Internet:")
print(state_summary[cols_show].head(5).to_string(index=False))

print("\nTop 5 States with Lowest % of Households Without Internet:")
print(state_summary[cols_show].tail(5).to_string(index=False))


Missing needed columns: {'hh_total_inet'}


KeyError: "You're missing {'hh_total_inet'}. Go back to the data-pull cell and include ACS codes B28002_013E (no_internet) AND B28002_001E (hh_total_inet). Re-run the pull, then re-run this cell."

### 4. Modeling and Evaluation
---
*(Describe and present the analytic models built on the data and evaluate the performance of the models for solving the problem)*

### 5. Conclusion
---
*(Briefly describe what you have done and what you discovered. Discuss any shortcomings of the process and results. Propose future work. **Finally, discuss the lessons learned from doing the project**.)*

### 6. References

---
# Use the following requirements for preparing your project:

## DO NOT DELETE THE CELLS BELLOW

# Project Requirements

This final project examines the level of knowledge the students have learned from the course. The following course outcomes will be checked against the content of the report:

Upon successful completion of this course, a student will be able to:
* observe and explore a variety of quantitative methods for data analysis;
* understand methods’ evaluation techniques to interpret their output;
* implement and evaluate methods to gain technical experience with data; and
* reproducibly execute an analytic project and represent/communicate its results faithfully.

** Marking will be foucsed on both presentation and content.**

## Written Presentation Requirements
The report will be judged on the basis of visual appearance, grammatical correctness, and quality of writing, as well as its contents. Please make sure that the text of your report is well-structured, using paragraphs, full sentences, and other features of well-written presentation.

## Technical Content:
* Is the problem well defined and described thoroughly?
* Is the size and complexity of the data set used in this project comparable to that of the example data sets used in the lectures and assignments?
* Did the report describe the charactriatics of the data?
* Did the report describe the goals of the data analysis?
* Did the analysis conduct exploratory analyses on the data?
* Did the analysis build analysis models of the data and evaluated the performance of the models?
* Overall, what is the rating of this project?